In [1]:
#Mounting gdrive if working on gdrive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(16)
from gensim import corpora, models
from pprint import pprint

In [5]:
 import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:

# Load Data
project_home = "/content/gdrive/My Drive/USML Project"
records_data = pd.read_csv(os.path.join(project_home, "all_trials.csv"))
mesh_data = pd.read_csv(os.path.join(project_home, "MESH_Terms_v2.csv"))
punc ='!"#$%&\'()*+,./:;<=>?@[\\]^`{|}~'

In [9]:
# Process clinical records data
records_data = records_data[['id','title','brief_summary','detailed_description']] # Subset columns
records_data = records_data.replace(np.nan, '', regex=True) # Replace Null values with empty strings
records_data['article'] = records_data['title'].str.cat(records_data['brief_summary'], sep=" ").str.cat(records_data['detailed_description'], sep=" ") # Create Concatenated text using all data
records_data = records_data.drop(['title','brief_summary','detailed_description'], axis=1)
records_data['article'] = records_data['article'].str.lower()   ## lower case
records_data.article =  records_data.article.str.translate(str.maketrans('','',punc))   # Remove punctuations except '-', '_'
records_data.article =  records_data.article.str.strip()    # Remove whitespaces


In [10]:
records_data[:10]

,id,article
0,NCT03260985,precision psychiatry continuity clinic project...
1,NCT03268473,effect of non-surgical periodontal treatment o...
2,NCT03262610,expanded-access for the use of setemelanotide ...
3,NCT03264261,constraint induced movement therapy for walkin...
4,NCT03267797,intrauterine administration of autologous hcg-...
5,NCT03260946,the use of complementary medicine in palliativ...
6,NCT03269786,laoboratory and ultrasonographic changes after...
7,NCT03260205,a phase 3 randomized double-blind multicenter ...
8,NCT03263650,randomized phase ii study of olaparib maintena...
9,NCT03269396,a prospective nonrandomized study of laryngeal...


In [11]:
# Lemmatizing function for inputted text.
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [12]:
# Preprocessing steps after tokenization of document text.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [13]:

print('Loading Snowball Stemmer into memory.')
stemmer = SnowballStemmer('english')

Loading Snowball Stemmer into memory.


In [14]:
processed_docs = records_data['article'].map(preprocess)

In [16]:
processed_docs[:10]

0    [precis, psychiatri, continu, clinic, project,...
1    [effect, surgic, periodont, treatment, oxid, s...
2    [expand, access, setemelanotid, singl, patient...
3    [constraint, induc, movement, therapi, walk, i...
4    [intrauterin, administr, autolog, activ, perip...
5    [complementari, medicin, care, centr, lyon, fr...
6    [laoboratori, chang, endoscop, varisc, treatme...
7    [phase, random, doubl, blind, multicent, paral...
8    [random, phase, studi, olaparib, mainten, foll...
9    [prospect, nonrandom, studi, laryng, allograft...
Name: article, dtype: object

In [17]:
#Create Bag of Words
dictionary = gensim.corpora.Dictionary(processed_docs)

In [18]:
#View the dictionary
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 appli
1 assess
2 care
3 clinic
4 conclus
5 continu
6 decis
7 demonstr
8 effect
9 evalu
10 explor


In [21]:
#Filter out the extremities
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [22]:
len(dictionary)

24775

In [23]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [26]:
test=bow_corpus[200000]


In [32]:
for i in range(len(test)):
    print((test[i][0], dictionary[test[i][0]],test[i][1]))


(2, 'care', 1)
(3, 'clinic', 1)
(25, 'outcom', 1)
(36, 'set', 1)
(76, 'object', 1)
(97, 'treatment', 6)
(103, 'despit', 1)
(121, 'sever', 1)
(122, 'singl', 1)
(127, 'collect', 2)
(129, 'conduct', 1)
(131, 'data', 2)
(135, 'follow', 3)
(147, 'post', 1)
(161, 'undergo', 2)
(162, 'visit', 1)
(164, 'week', 3)
(195, 'enrol', 1)
(230, 'observ', 1)
(259, 'year', 2)
(263, 'characterist', 1)
(361, 'mainten', 1)
(370, 'particip', 1)
(374, 'procedur', 3)
(389, 'standard', 1)
(390, 'start', 1)
(395, 'take', 1)
(427, 'option', 1)
(429, 'prospect', 1)
(452, 'experi', 1)
(482, 'primari', 2)
(520, 'center', 1)
(537, 'date', 2)
(586, 'offer', 1)
(609, 'remain', 1)
(665, 'label', 1)
(671, 'medic', 1)
(680, 'open', 1)
(692, 'sit', 1)
(727, 'recruit', 1)
(829, 'demograph', 1)
(878, 'proport', 1)
(1014, 'complet', 1)
(1080, 'endpoint', 1)
(1084, 'global', 2)
(1201, 'period', 2)
(1372, 'real', 2)
(1423, 'world', 2)
(1959, 'approxim', 1)
(2136, 'asthma', 2)
(2316, 'exacerb', 1)
(2507, 'symptomat', 1)
(2810, 

In [33]:
#Apply TF-IDF oin the corpus
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [35]:
len(corpus_tfidf)

345092

In [37]:
for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.08527146848753085), (1, 0.15043412904256084), (2, 0.10344577646577482), (3, 0.10766584752082192), (4, 0.13134472972911782), (5, 0.06123126926612932), (6, 0.10112047292689721), (7, 0.06989223062952193), (8, 0.02193960522638496), (9, 0.02313670025803864), (10, 0.08492130068382295), (11, 0.12317203920189113), (12, 0.08453739613866441), (13, 0.04913367677791193), (14, 0.0689140602355573), (15, 0.06684411196817518), (16, 0.08701166469691637), (17, 0.035672468286213184), (18, 0.08643156552624488), (19, 0.11003886114099984), (20, 0.13292861770255532), (21, 0.10113112242897473), (22, 0.031025254349011096), (23, 0.05931810939375318), (24, 0.7663631483416522), (25, 0.09175633982148544), (26, 0.07172225055126039), (27, 0.07244126895671481), (28, 0.1544744757278883), (29, 0.1249625358134768), (30, 0.14788561689853016), (31, 0.04973512713636691), (32, 0.12596240353976665), (33, 0.1804050628421558), (34, 0.13673404099384495), (35, 0.05097387344214872), (36, 0.07849821831131971), (37, 0.129595

In [ ]:
#Training LDA model using the Bag of Words:
lda_model = gensim.models.LdaMulticore(bow_corpus, id2word=dictionary, passes=2, workers=2)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, passes=2, workers=4)